# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [149]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: Get Data From Sheetsu

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very powerful way to share a dataset with colleagues as well as to create a mini centralized data storage, that is simpler to edit than a database.

A Google Spreadsheet with Wine data can be found [here](https://docs.google.com/spreadsheets/d/1mZ3Otr5AV4v8WwvLOAvWf3VLxDa-IeJ1AVTEuavJqeo/).

It can be accessed through sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/dab55afd

Questions:

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?
> Answer: it's a json string
- Check the status code of the response object. What code is it?
> 200
- Use the appropriate libraries and read functions to read the response into a Pandas Dataframe
> Possible answers include: pd.read_json and json.loads + pd.Dataframe
- Once you've imported the data into a dataframe, check the value of the 5th line: what's the price?
> 6

In [150]:
# You can either post or get info from this API
api_base_url = 'https://sheetsu.com/apis/v1.0/dab55afd'

In [151]:
# What kind of data is this returning?
api_response = requests.get(api_base_url)
api_response.text[:100]

u'[{"Color":"W","Region":"Portugal","Country":"Portugal","Vintage":"2013","Vinyard":"Vinho Verde","Nam'

In [152]:
reponse = json.loads(api_response.text)

In [153]:
type(reponse)

list

In [154]:
reponse[0]

{u'Color': u'W',
 u'Consumed In': u'2015',
 u'Country': u'Portugal',
 u'Grape': u'',
 u'Name': u'',
 u'Price': u'',
 u'Region': u'Portugal',
 u'Score': u'4',
 u'Vintage': u'2013',
 u'Vinyard': u'Vinho Verde'}

In [155]:
api_response.status_code

200

#### Lets read the data into a DataFrame!

In [156]:
wine_df = pd.DataFrame(reponse)
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland


#### Pandas has great functions. We could just do it this way

This sometimes works, but the data may need adjusting

In [157]:
wine_df = pd.read_json(api_response.text)
wine_df.head(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet


### Exercise 2: Post Data to Sheetsu
Now that we've learned how to read data, it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the following data to the spreadsheet:
- What status did you get? How can you check that you actually added the data correctly?
> Answer: send a get request and check the last line added
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?
> There will be many duplicate lines on the spreadsheet. One way to mitigate this would be through permission, another would be to insert at a specific position, so that the line is overwritten at each time.


In [158]:
post_data = {
'Grape' : ''
, 'Name' : 'My wonderful wine'
, 'Color' : 'R'
, 'Country' : 'US'
, 'Region' : 'Sonoma'
, 'Vinyard' : ''
, 'Score' : '10'
, 'Consumed In' : '2015'
, 'Vintage' : '1973'
, 'Price' : '200'
}

In [159]:
requests.post(api_base_url, data=post_data)

<Response [201]>

## Exercise 3: Data munging

Get back to the dataframe you've created in the beginning. Let's do some data munging:

1. Search for missing data
    - Is there any missing data? How do you deal with it?
    - Is there any data you can just remove?
    - Are the data types appropriate?
- Summarize the data 
    - Try using describe, min, max, mean, var

In [160]:
wine_df.head(1)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde


In [161]:
wine_df = wine_df.replace('', np.nan)

In [162]:
wine_df.head(1)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4,2013,Vinho Verde


In [163]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 10 columns):
Color          100 non-null object
Consumed In    100 non-null object
Country        98 non-null object
Grape          22 non-null object
Name           91 non-null object
Price          94 non-null object
Region         99 non-null object
Score          99 non-null object
Vintage        100 non-null object
Vinyard        35 non-null object
dtypes: object(10)
memory usage: 8.6+ KB


In [164]:
#wine_df[['Score', 'Price']] = wine_df[['Score', 'Price']].astype(float)

In [165]:
wine_df.describe()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
count,100,100,98,22,91,94,99,99,100,35
unique,5,9,7,16,36,20,22,10,11,30
top,R,2015,US,sauvignon blanc,My wonderful wine,200,Sonoma,10,1973,Top of the line wine
freq,80,81,80,4,44,67,64,66,63,3


In [166]:
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4,2013,Vinho Verde
1,W,2015,France,NaN,NaN,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,NaN,NaN,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,NaN,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,NaN,3,2012,Heartland


## Exercise 4: Feature Extraction

We would like to use a regression tree to predict the score of a wine. In order to do that, we first need to select and engineer appropriate features.

- Set the target to be the Score column, drop the rows with no score
- Use pd.get_dummies to create dummy features for all the text columns
- Fill the nan values in the numerical columns, using an appropriate method
- Train a Decision tree regressor on the Score, using a train test split:
        X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)
- Plot the test values, the predicted values and the residuals
- Calculate R^2 score
- Discuss your findings


In [167]:
dropped = wine_df[~wine_df['Score'].isnull()]

In [168]:
y = dropped['Score']

In [169]:
numerical = ['Consumed In', 'Price', 'Vintage']
dummies = [c for c in dropped.columns if c not in numerical + ['Score']]

In [170]:
X = dropped[numerical].join(pd.get_dummies(dropped[dummies]))

In [171]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.pipeline import make_pipeline

In [172]:
dt = make_pipeline(Imputer(),
                   DecisionTreeRegressor())

In [173]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)

In [174]:
#dt.fit(X_train, y_train)

In [175]:
#y_pred = dt.predict(X_test)

In [176]:
#plt.plot(y_test.values, 'o')
#plt.plot(y_pred, 'o')
#plt.legend(['actual', 'predicted'])

In [177]:
#resid = y_pred - y_test.values
#plt.plot(resid, 'o-')
#plt.axhline(resid.mean(), color = 'r')

In [178]:
#from sklearn.metrics import r2_score

In [179]:
#r2_score(y_test, y_pred)

> Discussion
The regression score is very bad and the predicted values do not seem to reproduce the expected values. This is probably due to 2 factors:
- too few data points to actually use machine learning
- features are probably not indicative of the score
Discuss with students here about the importance of not applying a ML model blindly.

## Exercise 5: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 5.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

In [180]:
#So this function gets a list of all 250 movies IDs.
#However, IMDB.com doesn't like people getting all of their data very easily, so we'll just use this page to get the IDs
#So this function doesn't iterate through pages, as all movies in the Top 250 are on a single page.
#It takes their unique IDs that are encoded in the HTML, and puts them in a list, called 'entries
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/top')
    html = response.text
    entries = re.findall("<a href.*?/title/(.*?)/", html) #Wrong regex
    return list(set(entries))

In [181]:
entries = get_top_250()

In [182]:
len(entries)

250

In [183]:
entries[0]

u'tt2582802'

## 5.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
1. Check the documentation of omdbapi.com to learn how to request movie data by id
- Define a function that returns a python object with all the information for a given id
- Iterate on all the IDs and store the results in a list of such objects
- Create a Pandas Dataframe from the list

In [184]:
#Now that we have the 250 IDs, we need a way to search omdapi (which has gathered all data for each IMDB movie in a 
#nice little JSON tree). 

#So we need to scrape each movie's JSON tree with Beautiful soup
#Just like with indeed.com, it's going to use omdabpi's search engine 250 times, once for each id in the entries list
#from above. After it searches a movie id in the lsit above, it will scrape its JSON tree.
def get_entry(entry):
    res = requests.get('http://www.omdbapi.com/?i='+entry)
    if res.status_code != 200:
        print entry, res.status_code
    else:
        print '.',
    try:
        j = json.loads(res.text)
    except ValueError:
        j = None
    return j

In [185]:
#So you're going to repreat the function above for every item(movie id) in the 'entries' list
#It returns a dictionary that can then be turned into a dataframe
entries_dict_list = [get_entry(e) for e in entries]

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [186]:
len(entries_dict_list)

250

In [187]:
#Here we turn the JSON file for each of th 250 movies into a dataframe
df = pd.DataFrame(entries_dict_list)

In [188]:
df.head()

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",Won 3 Oscars. Another 87 wins & 128 nominations.,USA,Damien Chazelle,"Drama, Music",English,88,A promising young drummer enrolls at a cut-thr...,http://ia.media-imdb.com/images/M/MV5BMTU4OTQ3...,R,15 Oct 2014,True,107 min,Whiplash,movie,Damien Chazelle,2014,tt2582802,8.5,"384,504"
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Nominated for 2 Oscars. Another 5 wins & 6 nom...,Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,99,A poor village under attack by bandits recruit...,http://ia.media-imdb.com/images/M/MV5BMTc5MDY1...,UNRATED,19 Nov 1956,True,207 min,Seven Samurai,movie,"Akira Kurosawa (screenplay), Shinobu Hashimoto...",1954,tt0047478,8.7,"226,364"
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Won 4 Oscars. Another 29 wins & 23 nominations.,USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",85,Archaeologist and adventurer Indiana Jones is ...,http://ia.media-imdb.com/images/M/MV5BMjA0ODEz...,PG,12 Jun 1981,True,115 min,Raiders of the Lost Ark,movie,"Lawrence Kasdan (screenplay), George Lucas (st...",1981,tt0082971,8.5,"653,557"
3,"William Holden, Alec Guinness, Jack Hawkins, S...",Won 7 Oscars. Another 23 wins & 7 nominations.,"UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",N/A,After settling his differences with a Japanese...,http://ia.media-imdb.com/images/M/MV5BMTc2NzA0...,PG,14 Dec 1957,True,161 min,The Bridge on the River Kwai,movie,"Pierre Boulle (novel), Carl Foreman (screenpla...",1957,tt0050212,8.2,"147,591"
4,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",Nominated for 1 Oscar. Another 36 wins & 78 no...,USA,Joss Whedon,"Action, Adventure, Sci-Fi","English, Russian",69,Earth's mightiest heroes must come together an...,http://ia.media-imdb.com/images/M/MV5BMTk2NTI1...,PG-13,04 May 2012,True,143 min,The Avengers,movie,"Joss Whedon (screenplay), Zak Penn (story), Jo...",2012,tt0848228,8.1,"980,989"


## 5.c Get gross data

The OMDB API is great, but it does not provide information about Gross Revenue of the movie. We'll revert back to scraping for this.

1. Write a function that retrieves the gross revenue from the entry page at imdb.com
- The function should handle the exception of when the page doesn't report gross revenue
- Retrieve the gross revenue for each movie and store it in a separate dataframe

In [189]:
#There is still some information we would want, but OMDb API does not provide
#So, we have to go back to imdb.com to scrape the gross revenue for each movie
#This function will ultimately search for each movie by their id in the entries list, and scrape the gross revenue into
#a new list called 'grosses

def get_gross(entry): #define the function
    response = requests.get('http://www.imdb.com/title/'+entry) #This will generate a request from the page for an entry
    html = response.text
    try:
        gross_list = re.findall("Gross:</h4>[ ]*\$([^ ]*)", html) #This will create a list with the value after the word 'Gross'
        gross = int(gross_list[0].replace(',', '')) #This creates a new value by convertinf the above to an integer and eliminating commas
        print '.',
        return gross
    except Exception as ex:
        print
        print ex, entry, response.status_code
        return None

In [190]:
grosses = [(e, get_gross(e)) for e in entries]#Repeat the function above for each id in the entries list

. . . . . . .
list index out of range tt0046268 200

list index out of range tt0055630 200
. . . . .
list index out of range tt0057115 200
.
list index out of range tt0071315 200
. . . . .
list index out of range tt0074896 200
. .
list index out of range tt0021749 200
. .
list index out of range tt0050613 200

list index out of range tt0053125 200
. . .
list index out of range tt1220719 200

list index out of range tt0025316 200
. .
list index out of range tt0083922 200
. . .
list index out of range tt0072684 200
.
list index out of range tt0074958 200
. . . .
list index out of range tt0036775 200
. . . . . .
list index out of range tt0978762 200
. .
list index out of range tt0080678 200
. .
list index out of range tt0056592 200
.
list index out of range tt0095327 200
. . . . . . . . .
list index out of range tt0046438 200
.
list index out of range tt0015864 200
.
list index out of range tt0045152 200
. . . .
list index out of range tt0046911 200
.
list index out of range tt0050986 200

In [217]:
#now we have a second dataframe with the id and gross for each  title
#we'll merge this with the dataframe above, which also includes the imdbID for each movie
#then we can actually do some shit
df1 = pd.DataFrame(grosses, columns=['imdbID', 'Gross'])

,imdbID,Gross
0,tt2582802,13092000.0
1,tt0047478,269061.0
2,tt0082971,242374454.0
3,tt0050212,27200000.0
4,tt0848228,623279547.0
5,tt1663202,183635922.0
6,tt0338564,89594.0
7,tt0046268,NaN
8,tt0055630,NaN
9,tt0266543,380838870.0


In [192]:
df1.head

(250, 2)

In [193]:
df.shape

(250, 20)

## 5.d Data munging

1. Now that you have movie information and gross revenue information, let's clean the two datasets.
- Check if there are null values. Be careful they may appear to be valid strings.
- Convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Merge the data from the two datasets into a single one

In [195]:
df = df.replace('N/A', np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 20 columns):
Actors        250 non-null object
Awards        245 non-null object
Country       250 non-null object
Director      250 non-null object
Genre         250 non-null object
Language      249 non-null object
Metascore     169 non-null object
Plot          250 non-null object
Poster        250 non-null object
Rated         249 non-null object
Released      249 non-null object
Response      250 non-null object
Runtime       250 non-null object
Title         250 non-null object
Type          250 non-null object
Writer        250 non-null object
Year          250 non-null object
imdbID        250 non-null object
imdbRating    250 non-null object
imdbVotes     250 non-null object
dtypes: object(20)
memory usage: 39.1+ KB


In [196]:
df.Released = pd.to_datetime(df.Released)

In [197]:
def intminutes(x):
    y = x.replace('min', '').strip()
    return int(y)

df.Runtime = df.Runtime.apply(intminutes)

In [198]:
df.Year = df.Year.astype(int)

In [199]:
df.imdbRating = df.imdbRating.astype(float)

In [200]:
def intvotes(x):
    y = x.replace(',', '').strip()
    return int(y)
df.imdbVotes = df.imdbVotes.apply(intvotes)

In [202]:
df = pd.merge(df, df1)

In [203]:
df.head()

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",Won 3 Oscars. Another 87 wins & 128 nominations.,USA,Damien Chazelle,"Drama, Music",English,88,A promising young drummer enrolls at a cut-thr...,http://ia.media-imdb.com/images/M/MV5BMTU4OTQ3...,R,...,True,107,Whiplash,movie,Damien Chazelle,2014,tt2582802,8.5,384504,13092000.0
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Nominated for 2 Oscars. Another 5 wins & 6 nom...,Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,99,A poor village under attack by bandits recruit...,http://ia.media-imdb.com/images/M/MV5BMTc5MDY1...,UNRATED,...,True,207,Seven Samurai,movie,"Akira Kurosawa (screenplay), Shinobu Hashimoto...",1954,tt0047478,8.7,226364,269061.0
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Won 4 Oscars. Another 29 wins & 23 nominations.,USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",85,Archaeologist and adventurer Indiana Jones is ...,http://ia.media-imdb.com/images/M/MV5BMjA0ODEz...,PG,...,True,115,Raiders of the Lost Ark,movie,"Lawrence Kasdan (screenplay), George Lucas (st...",1981,tt0082971,8.5,653557,242374454.0
3,"William Holden, Alec Guinness, Jack Hawkins, S...",Won 7 Oscars. Another 23 wins & 7 nominations.,"UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",NaN,After settling his differences with a Japanese...,http://ia.media-imdb.com/images/M/MV5BMTc2NzA0...,PG,...,True,161,The Bridge on the River Kwai,movie,"Pierre Boulle (novel), Carl Foreman (screenpla...",1957,tt0050212,8.2,147591,27200000.0
4,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",Nominated for 1 Oscar. Another 36 wins & 78 no...,USA,Joss Whedon,"Action, Adventure, Sci-Fi","English, Russian",69,Earth's mightiest heroes must come together an...,http://ia.media-imdb.com/images/M/MV5BMTk2NTI1...,PG-13,...,True,143,The Avengers,movie,"Joss Whedon (screenplay), Zak Penn (story), Jo...",2012,tt0848228,8.1,980989,623279547.0


## 5.d Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to modify the `token_pattern` in the `CountVectorizer`.

In [204]:
from sklearn.feature_extraction.text import CountVectorizer

In [205]:
#You take the genre column and pull out each string, and turn it into a binary value (either the movie it that genre or not)
cv = CountVectorizer()
data = cv.fit_transform(df.Genre).todense()
columns = ['genre_'+c for c in cv.get_feature_names()]
genredf = pd.DataFrame(data, columns=columns)
genredf.head()

,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_drama,genre_family,genre_fantasy,genre_fi,...,genre_music,genre_musical,genre_mystery,genre_noir,genre_romance,genre_sci,genre_sport,genre_thriller,genre_war,genre_western
0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [206]:
#Now you add that dataframe of genre binary values to the original dataframe
df = pd.concat([df, genredf], axis = 1)

In [207]:
#repeat the process for actors
cv = CountVectorizer(token_pattern=u'(?u)\\w+\.?\\w?\.? \\w+')
data = cv.fit_transform(df.Actors).todense()
columns = ['actor: '+c for c in cv.get_feature_names()]
actorsdf = pd.DataFrame(data, columns=columns)
actorsdf.head()

,actor: aamir khan,actor: aaron eckhart,actor: abdel ahmed,actor: adam baldwin,actor: adam driver,actor: adolphe menjou,actor: adrien brody,actor: agnes moorehead,actor: ahney her,actor: akemi yamaguchi,...,actor: woody harrelson,actor: xolani mali,actor: yacef saadi,actor: yoshiko shinohara,actor: yukiko shimazaki,actor: yves montand,actor: yôko tsukasa,actor: zach grenier,actor: zoe saldana,actor: álvaro guerrero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
df.Actors[0]

u'Miles Teller, J.K. Simmons, Paul Reiser, Melissa Benoist'

In [209]:
actorsdf.loc[0,actorsdf.iloc[0] != 0]

actor: j.k. simmons       1
actor: melissa benoist    1
actor: miles teller       1
actor: paul reiser        1
Name: 0, dtype: int64

In [210]:
#Now you have a shit ton of rows for each of the 250 movies
df = pd.concat([df, actorsdf], axis = 1)

## Bonus:

1. What are the top 10 grossing movies?
- Who are the 10 actors that appear in the most movies?
- What's the average grossing of the movies in which each of these actors appear?
- What genre is the oldest movie?


In [211]:
df.columns

Index([                  u'Actors',                   u'Awards',
                        u'Country',                 u'Director',
                          u'Genre',                 u'Language',
                      u'Metascore',                     u'Plot',
                         u'Poster',                    u'Rated',
       ...
         u'actor: woody harrelson',       u'actor: xolani mali',
             u'actor: yacef saadi', u'actor: yoshiko shinohara',
        u'actor: yukiko shimazaki',      u'actor: yves montand',
            u'actor: yôko tsukasa',      u'actor: zach grenier',
             u'actor: zoe saldana',   u'actor: álvaro guerrero'],
      dtype='object', length=855)

In [212]:
df[['Title','Gross', 'Genre']].sort_values('Gross', ascending = False).head(10)

,Title,Gross,Genre
161,Star Wars: The Force Awakens,936627416.0,"Action, Adventure, Fantasy"
4,The Avengers,623279547.0,"Action, Adventure, Sci-Fi"
39,The Dark Knight,533316061.0,"Action, Adventure, Crime"
235,Star Wars: Episode IV - A New Hope,460935665.0,"Action, Adventure, Fantasy"
160,The Dark Knight Rises,448130642.0,"Action, Adventure, Drama"
50,The Lion King,422783777.0,"Animation, Adventure, Drama"
101,Toy Story 3,414984497.0,"Animation, Adventure, Comedy"
229,Captain America: Civil War,406226381.0,"Action, Adventure, Sci-Fi"
70,Harry Potter and the Deathly Hallows: Part 2,380955619.0,"Adventure, Drama, Fantasy"
9,Finding Nemo,380838870.0,"Animation, Adventure, Comedy"


In [213]:
actorcols = actorsdf.columns

In [214]:
#This creates a dataframe of top actors, sorted by the sum of times they're present in the top 250, and it only includes
#the Top 10
topactors = actorsdf.sum().sort_values(ascending = False).head(10)
topactors

actor: leonardo dicaprio    7
actor: harrison ford        7
actor: robert de            7
actor: tom hanks            6
actor: clint eastwood       6
actor: tom hardy            5
actor: mark hamill          5
actor: brad pitt            4
actor: morgan freeman       4
actor: charles chaplin      4
dtype: int64

In [215]:
#This is a dataframe that gives the average gross for a movie in which our top actors above starred in
means = [df.loc[df[actor]==1,'Gross'].mean() for actor in topactors.index]
means = pd.Series(means, index = topactors.index)
means.sort_values(ascending = False)

actor: mark hamill          4.992118e+08
actor: harrison ford        3.519134e+08
actor: tom hanks            2.423047e+08
actor: tom hardy            2.183233e+08
actor: leonardo dicaprio    1.686647e+08
actor: morgan freeman       8.251176e+07
actor: brad pitt            7.194123e+07
actor: clint eastwood       7.185320e+07
actor: robert de            3.655932e+07
actor: charles chaplin      1.331622e+06
dtype: float64

In [216]:
df.sort_values('Released')[['Genre', 'Title', 'Released', 'Gross']].head()

,Genre,Title,Released,Gross
89,"Comedy, Drama, Family",The Kid,1921-02-06,2500000.0
180,"Action, Adventure, Comedy",The General,1927-02-24,NaN
211,"Drama, Sci-Fi",Metropolis,1927-03-13,26435.0
194,"Drama, Romance",Sunrise,1927-11-04,NaN
25,"Comedy, Drama, Romance",City Lights,1931-03-07,NaN
